In [1]:
import numpy as np
import os
import pandas as pd

from glob import glob

os.chdir("../")

In [2]:
year = 2021
election_round = "first_round" # first_round, runoff

In [3]:
if year == 2013:
    output = []
    for file in glob(f"data/Chile/{year}_{election_round}_*.xlsx"):
        region_id = file.replace(f"data/Chile\{year}_{election_round}_", "")[0:2]
        tmp = pd.read_excel(file)
        tmp["region_id"] = region_id
        output.append(tmp)

    df = pd.concat(output, ignore_index=True)
elif year in [2017, 2021]:
    df = pd.read_excel(f"data/Chile/{year}_{election_round}.xlsx")

df = df.rename(columns={
    "Candidato": "candidate", 
    "Votos TRICEL": "value", 
    "Nro. Mesa": "mesa", 
    "Tipo de mesa": "tipo mesa",
    "Circ.Electoral": "circ. electoral"
})
df["candidate"] = df["candidate"].str.upper().str.strip()
df["value"] = df["value"].fillna(0)

df.columns = [x.lower().strip() for x in df.columns]

df.loc[df["provincia"] == "ÑUBLE", "region_id"] = 16
df.loc[df["provincia"] == "ÑUBLE", "región"] = 16

df = df.dropna(subset=["comuna"])

df["candidate"] = df["candidate"].replace({
    "VOTOS EN BLANCO": "BLANK",
    "VOTOS BLANCOS": "BLANK",
    "VOTOS NULOS": "NULL"
})

def create_polling_id(data, columns=[]):
    output = data[columns[0]].fillna("###").copy()

    for column in columns[1:]:
        output += "-" + data[column].fillna("###").astype(str)
    return output

if year == 2013:
    df["polling_id"] = create_polling_id(df, columns=["comuna", "circ. electoral", "mesa", "tipo mesa"])

elif year in [2017, 2021]:
    df["polling_id"] = create_polling_id(df, columns=["comuna", "circ. electoral", "mesa", "tipo mesa", "mesas fusionadas"])

df.head()

,nro. región,región,provincia,circ. senatorial,distrito,comuna,circ. electoral,local,mesa,tipo mesa,mesas fusionadas,electores,nro. en voto,candidate,value,unnamed: 15,region_id,polling_id
0,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,1.0,GABRIEL BORIC FONT,22.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
1,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,2.0,JOSE ANTONIO KAST RIST,36.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
2,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,3.0,YASNA PROVOSTE CAMPILLAY,15.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
3,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,4.0,SEBASTIAN SICHEL RAMIREZ,12.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
4,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,5.0,EDUARDO ARTES BRICHETTI,1.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M


In [4]:
df

,nro. región,región,provincia,circ. senatorial,distrito,comuna,circ. electoral,local,mesa,tipo mesa,mesas fusionadas,electores,nro. en voto,candidate,value,unnamed: 15,region_id,polling_id
0,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,1.0,GABRIEL BORIC FONT,22.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
1,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,2.0,JOSE ANTONIO KAST RIST,36.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
2,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,3.0,YASNA PROVOSTE CAMPILLAY,15.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
3,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,4.0,SEBASTIAN SICHEL RAMIREZ,12.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
4,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,5.0,EDUARDO ARTES BRICHETTI,1.0,NaN,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419746,16.0,DE ÑUBLE,DIGUILLIN,16.0,19.0,YUNGAY,YUNGAY,LICEO DE YUNGAY,21.0,M,21M,340.0,5.0,EDUARDO ARTES BRICHETTI,1.0,NaN,NaN,YUNGAY-YUNGAY-21.0-M-21M
419747,16.0,DE ÑUBLE,DIGUILLIN,16.0,19.0,YUNGAY,YUNGAY,LICEO DE YUNGAY,21.0,M,21M,340.0,6.0,MARCO ENRIQUEZ-OMINAMI GUMUCIO,10.0,NaN,NaN,YUNGAY-YUNGAY-21.0-M-21M
419748,16.0,DE ÑUBLE,DIGUILLIN,16.0,19.0,YUNGAY,YUNGAY,LICEO DE YUNGAY,21.0,M,21M,340.0,7.0,FRANCO PARISI FERNANDEZ,30.0,NaN,NaN,YUNGAY-YUNGAY-21.0-M-21M
419749,16.0,DE ÑUBLE,DIGUILLIN,16.0,19.0,YUNGAY,YUNGAY,LICEO DE YUNGAY,21.0,M,21M,340.0,900.0,NULL,1.0,NaN,NaN,YUNGAY-YUNGAY-21.0-M-21M


#### Abstentions
The election data from 2017 and 2021 includes as a feature the number of expected electors by polling station. We flag the data, by including "Abstentions" as a candidate.

In [5]:
df_abstentions = pd.DataFrame()
if year in [2017, 2021]:
    df_abstentions = df.groupby("polling_id").agg({"value": "sum"}).reset_index()
    df_abstentions = pd.merge(
        df[["polling_id", "electores"]].drop_duplicates(), 
        df_abstentions, 
        on="polling_id"
    )
    df_abstentions["value2"] = df_abstentions["electores"] - df_abstentions["value"]
    df_abstentions = df_abstentions[["polling_id", "value2"]].rename(columns={"value2": "value"})
    df_abstentions["candidate"] = "ABSTENTION"
    df_abstentions["flag_candidates"] = 0

#### Spoilt votes

In [6]:
df_spoilt = df[df["candidate"].isin(["NULL", "BLANK"])].copy()
df_spoilt = df_spoilt[["candidate", "value", "polling_id"]]
df_spoilt["flag_candidates"] = 0
df_spoilt.head()

,candidate,value,polling_id,flag_candidates
7,NULL,1.0,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M,0
8,BLANK,1.0,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M,0
16,NULL,0.0,ALTO HOSPICIO-ALTO HOSPICIO-21.0-M-21M,0
17,BLANK,0.0,ALTO HOSPICIO-ALTO HOSPICIO-21.0-M-21M,0
25,NULL,8.0,ALTO HOSPICIO-ALTO HOSPICIO-22.0-M-22M,0


In [7]:
df.candidate.unique()

array(['GABRIEL BORIC FONT', 'JOSE ANTONIO KAST RIST',
       'YASNA PROVOSTE CAMPILLAY', 'SEBASTIAN SICHEL RAMIREZ',
       'EDUARDO ARTES BRICHETTI', 'MARCO ENRIQUEZ-OMINAMI GUMUCIO',
       'FRANCO PARISI FERNANDEZ', 'NULL', 'BLANK'], dtype=object)

In [8]:
df = df[~df["candidate"].isin(["BLANK", "NULL", "TOTALES", np.nan])]

In [9]:
df = df.dropna(axis=1, how="all")



In [10]:
df_filtered = df.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
df_filtered["rate"] = df_filtered.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
df_filtered = df_filtered.reset_index()
df_filtered["rank"] = df_filtered.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)
df_filtered["flag_candidates"] = 1
df_filtered.head()

,polling_id,candidate,value,rate,rank,flag_candidates
0,ALGARROBO-ALGARROBO-1.0-M-1M,EDUARDO ARTES BRICHETTI,1.0,0.006024,7,1
1,ALGARROBO-ALGARROBO-1.0-M-1M,FRANCO PARISI FERNANDEZ,14.0,0.084337,5,1
2,ALGARROBO-ALGARROBO-1.0-M-1M,GABRIEL BORIC FONT,29.0,0.174699,3,1
3,ALGARROBO-ALGARROBO-1.0-M-1M,JOSE ANTONIO KAST RIST,53.0,0.319277,1,1
4,ALGARROBO-ALGARROBO-1.0-M-1M,MARCO ENRIQUEZ-OMINAMI GUMUCIO,10.0,0.060241,6,1


In [11]:
data = pd.concat([
    df_filtered,
    df_abstentions,
    df_spoilt
])

data.head()

,polling_id,candidate,value,rate,rank,flag_candidates
0,ALGARROBO-ALGARROBO-1.0-M-1M,EDUARDO ARTES BRICHETTI,1.0,0.006024,7.0,1
1,ALGARROBO-ALGARROBO-1.0-M-1M,FRANCO PARISI FERNANDEZ,14.0,0.084337,5.0,1
2,ALGARROBO-ALGARROBO-1.0-M-1M,GABRIEL BORIC FONT,29.0,0.174699,3.0,1
3,ALGARROBO-ALGARROBO-1.0-M-1M,JOSE ANTONIO KAST RIST,53.0,0.319277,1.0,1
4,ALGARROBO-ALGARROBO-1.0-M-1M,MARCO ENRIQUEZ-OMINAMI GUMUCIO,10.0,0.060241,6.0,1


In [12]:
data.groupby("candidate").agg({"value": "sum"})

,value
candidate,
ABSTENTION,7879680.0
BLANK,30427.0
EDUARDO ARTES BRICHETTI,102209.0
FRANCO PARISI FERNANDEZ,899067.0
GABRIEL BORIC FONT,1796851.0
JOSE ANTONIO KAST RIST,1954607.0
MARCO ENRIQUEZ-OMINAMI GUMUCIO,533554.0
NULL,55316.0
SEBASTIAN SICHEL RAMIREZ,894838.0


In [13]:
print(df_filtered.shape)
df_filtered.head()

(326473, 6)


,polling_id,candidate,value,rate,rank,flag_candidates
0,ALGARROBO-ALGARROBO-1.0-M-1M,EDUARDO ARTES BRICHETTI,1.0,0.006024,7,1
1,ALGARROBO-ALGARROBO-1.0-M-1M,FRANCO PARISI FERNANDEZ,14.0,0.084337,5,1
2,ALGARROBO-ALGARROBO-1.0-M-1M,GABRIEL BORIC FONT,29.0,0.174699,3,1
3,ALGARROBO-ALGARROBO-1.0-M-1M,JOSE ANTONIO KAST RIST,53.0,0.319277,1,1
4,ALGARROBO-ALGARROBO-1.0-M-1M,MARCO ENRIQUEZ-OMINAMI GUMUCIO,10.0,0.060241,6,1


In [14]:
data.to_csv(f"data_output/Chile/{year}_{election_round}.csv.gz", compression="gzip", index=False)

In [15]:
list(data)

['polling_id', 'candidate', 'value', 'rate', 'rank', 'flag_candidates']

In [16]:
# df[["polling_id", "Región", "Provincia", "Circ.Senatorial", "Distrito", "Comuna", "Circ.Electoral", "Local",
#         "Mesa", "Tipo mesa", "Mesas Fusionadas", "Electores"]].drop_duplicates()

In [20]:
df

,nro. región,región,provincia,circ. senatorial,distrito,comuna,circ. electoral,local,mesa,tipo mesa,mesas fusionadas,electores,nro. en voto,candidate,value,unnamed: 15,polling_id
0,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,1.0,GABRIEL BORIC FONT,22.0,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
1,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,2.0,JOSE ANTONIO KAST RIST,36.0,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
2,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,3.0,YASNA PROVOSTE CAMPILLAY,15.0,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
3,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,4.0,SEBASTIAN SICHEL RAMIREZ,12.0,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
4,1.0,DE TARAPACA,IQUIQUE,2.0,2.0,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,20.0,M,20M,333.0,5.0,EDUARDO ARTES BRICHETTI,1.0,NaN,ALTO HOSPICIO-ALTO HOSPICIO-20.0-M-20M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419744,16.0,DE ÑUBLE,DIGUILLIN,16.0,19.0,YUNGAY,YUNGAY,LICEO DE YUNGAY,21.0,M,21M,340.0,3.0,YASNA PROVOSTE CAMPILLAY,20.0,NaN,YUNGAY-YUNGAY-21.0-M-21M
419745,16.0,DE ÑUBLE,DIGUILLIN,16.0,19.0,YUNGAY,YUNGAY,LICEO DE YUNGAY,21.0,M,21M,340.0,4.0,SEBASTIAN SICHEL RAMIREZ,30.0,NaN,YUNGAY-YUNGAY-21.0-M-21M
419746,16.0,DE ÑUBLE,DIGUILLIN,16.0,19.0,YUNGAY,YUNGAY,LICEO DE YUNGAY,21.0,M,21M,340.0,5.0,EDUARDO ARTES BRICHETTI,1.0,NaN,YUNGAY-YUNGAY-21.0-M-21M
419747,16.0,DE ÑUBLE,DIGUILLIN,16.0,19.0,YUNGAY,YUNGAY,LICEO DE YUNGAY,21.0,M,21M,340.0,6.0,MARCO ENRIQUEZ-OMINAMI GUMUCIO,10.0,NaN,YUNGAY-YUNGAY-21.0-M-21M


In [21]:
if year == 2021:

    df_location = df[["polling_id", "nro. región", "región", "provincia", "circ. senatorial", "distrito", "comuna", "circ. electoral", "local",
        "mesa", "tipo mesa", "mesas fusionadas", "electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "nro. región": "region_id",
        "región": "region",
        "provincia": "province",
        "circ. senatorial": "senate_district",
        "distrito": "district",
        "comuna": "commune",
        "circ. electoral": "electoral_circ",
        "local": "polling_place",
        "mesa": "polling_table",
        "tipo mesa": "polling_type",
        "mesas fusionadas": "mixed_polling",
        "electores": "voters"
    })
    
    for col in ["region_id", "senate_district", "district", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)
    
elif year == 2017:
    df_location = df[["polling_id", "región", "provincia", "circ.senatorial", "distrito", "comuna", "circ. electoral", "local",
        "mesa", "tipo mesa", "mesas fusionadas", "electores"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "región": "region_id",
        "provincia": "province",
        "circ.senatorial": "senate_district",
        "distrito": "district",
        "comuna": "commune",
        "circ.electoral": "electoral_circ",
        "local": "polling_place",
        "mesa": "polling_table",
        "tipo mesa": "polling_type",
        "mesas fusionadas": "mixed_polling",
        "electores": "voters"
    })
    
    for col in ["region_id", "polling_table", "voters"]:
        df_location[col] = df_location[col].astype(int)

elif year == 2013:
    df_location = df[["polling_id", "region_id", "provincia", "circ. senatorial", "distrito", "comuna", "circ. electoral", 
        "nro. mesa", "tipo mesa"]].drop_duplicates()

    df_location = df_location.rename(columns={
        "provincia": "province",
        "circ. senatorial": "senate_district",
        "distrito": "district",
        "comuna": "commune",
        "circ. electoral": "electoral_circ",
        "nro. mesa": "polling_table",
        "tipo mesa": "polling_type"
    })
    
    for col in ["region_id", "polling_table"]:
        df_location[col] = df_location[col].astype(int)

df_location.to_csv(f"data_output/Chile/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)